In [1]:
# import torch
# print(torch.__version__)
# print(torch.cuda.is_available())
# print(torch.backends.cudnn.enabled) # cudnn도 같이 설치됨
# print(torch.cuda.get_device_name(0))  # GPU 이름을 출력

In [3]:
#dir
repo_path_local = "zhengchong/CatVTON"
data_root_path = r'C:\Users\epdgn\Downloads\catvton_FASTAPI\trenbe'
output_dir = r'C:\Users\epdgn\Downloads\catvton_FASTAPI\output'

In [4]:
# masking
!python preprocess_agnostic_mask.py --data_root_path "{data_root_path}" --repo_path "{repo_path_local}"


Fetching 10 files: 100%|██████████| 10/10 [00:00<?, ?it/s]

Processing test:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\epdgn\anaconda3\envs\catvton\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Processing test: 100%|██████████| 1/1 [00:02<00:00,  2.25s/it]


In [5]:
!python -u inference.py \
    --dataset "trenbe" \
    --data_root_path "{data_root_path}" \
    --output_dir "{output_dir}" \
    --dataloader_num_workers 0 \
    --repaint \
    --batch_size 1 \
    --seed 555

Arguments: {'base_model_path': 'booksforcharlie/stable-diffusion-inpainting', 'resume_path': 'zhengchong/CatVTON', 'dataset_name': 'trenbe', 'data_root_path': 'C:\\Users\\epdgn\\Downloads\\catvton_FASTAPI\\trenbe', 'output_dir': 'C:\\Users\\epdgn\\Downloads\\catvton_FASTAPI\\output', 'seed': 555, 'batch_size': 1, 'num_inference_steps': 75, 'guidance_scale': 2.5, 'width': 384, 'height': 512, 'repaint': True, 'concat_eval_results': False, 'allow_tf32': True, 'dataloader_num_workers': 0, 'mixed_precision': 'bf16', 'concat_axis': 'y', 'enable_condition_noise': True}
Initializing CatVTONPipeline...
Downloaded zhengchong/CatVTON to C:\Users\epdgn\.cache\huggingface\hub\models--zhengchong--CatVTON\snapshots\f15e18d054cdd20d01249944a312c0c5d33fd8b4
CatVTONPipeline initialized successfully.
Loading pair file: C:\Users\epdgn\Downloads\catvton_FASTAPI\trenbe\test\test_pairs_paired.txt
Loaded 1 pairs from pair file.
InferenceDataset initialized with 1 items.
Batch loaded: dict_keys(['index', 'pers

c:\Users\epdgn\anaconda3\envs\catvton\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()

Fetching 10 files: 100%|██████████| 10/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  1%|▏         | 1/75 [00:00<00:31,  2.31it/s]

  3%|▎         | 2/75 [00:00<00:28,  2.53it/s]

  4%|▍         | 3/75 [00:01<00:27,  2.57it/s]

  5%|▌         | 4/75 [00:01<00:27,  2.62it/s]

  7%|▋         | 5/75 [00:01<00:26,  2.62it/s]

  8%|▊         | 6/75 [00:02<00:26,  2.62it/s]

  9%|▉         | 7/75 [00:02<00:26,  2.61it/s]

 11%|█         | 8/75 [00:03<00:25,  2.61it/s]

 12%|█▏        | 9/75 [00:03<00:25,  2.61it/s]

 13%|█▎        | 10/

In [19]:
from PIL import Image
import os
import numpy as np
import cv2
from model.cloth_masker import AutoMasker

# Initialize AutoMasker
automasker = AutoMasker()

# Define base paths
base_dir = r"C:\Users\epdgn\Downloads\catvton_test\trenbe\test"
output_dir = os.path.join(base_dir, "validation")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define subdirectories
original_dir = os.path.join(base_dir, "images")
cloth_dir = os.path.join(base_dir, "cloth")
prediction_dir = r"C:\Users\epdgn\Downloads\catvton_test\output\trenbe-512\result"
densepose_dir = os.path.join(base_dir, "images")
schp_lip_dir = os.path.join(base_dir, "images")
schp_atr_dir = os.path.join(base_dir, "images")
mask_dir = os.path.join(base_dir, "agnostic_masks")

# Standard size for all images
resize_width = 512
resize_height = 512

# Get all image filenames in the prediction directory
image_filenames = [f for f in os.listdir(prediction_dir) if f.endswith(".jpg")]

for file_name in image_filenames:
    base_file_name = os.path.splitext(file_name)[0]  # Remove extension

    # Define paths for each image type
    prediction_image_path = os.path.join(prediction_dir, file_name)
    densepose_image_path = os.path.join(densepose_dir, file_name)
    schp_lip_atr_path = os.path.join(schp_atr_dir, file_name)
    mask_path = os.path.join(mask_dir, f"{base_file_name}.png")
    original_image_path = os.path.join(original_dir, file_name)
    cloth_image_path = os.path.join(cloth_dir, file_name)

    # Open and resize the images
    prediction_image = Image.open(prediction_image_path).resize((resize_width, resize_height), Image.LANCZOS)
    original_image = Image.open(original_image_path).resize((resize_width, resize_height), Image.LANCZOS)
    cloth_image = Image.open(cloth_image_path).resize((resize_width, resize_height), Image.LANCZOS)

    # Use AutoMasker to process the images
    processed_images = automasker.preprocess_image(prediction_image_path)
    densepose_array = np.array(processed_images['densepose'])
    densepose_array = cv2.normalize(densepose_array, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    densepose_color = cv2.applyColorMap(densepose_array, cv2.COLORMAP_JET)
    densepose_image = Image.fromarray(cv2.cvtColor(densepose_color, cv2.COLOR_BGR2RGB)).resize((resize_width, resize_height), Image.LANCZOS)
    schp_lip_image = processed_images['schp_lip'].resize((resize_width, resize_height), Image.LANCZOS)
    schp_atr_image = processed_images['schp_atr'].resize((resize_width, resize_height), Image.LANCZOS)
    mask_image = Image.open(mask_path).resize((resize_width, resize_height), Image.LANCZOS)

    # Create a new blank image for 2 rows layout (top 3 images, bottom 4 images)
    combined_width = resize_width * 4  # Maximum width for 4 images in a row
    combined_height = resize_height * 2  # Maximum height for 2 rows
    combined_image = Image.new("RGB", (combined_width, combined_height))

    # Paste the images into the combined image
    # First row (original, cloth, prediction)
    combined_image.paste(original_image, (0, 0))
    combined_image.paste(cloth_image, (resize_width, 0))
    combined_image.paste(prediction_image, (resize_width * 2, 0))

    # Second row (densepose, schp_lip, schp_atr, mask)
    combined_image.paste(densepose_image, (0, resize_height))
    combined_image.paste(schp_lip_image, (resize_width, resize_height))
    combined_image.paste(schp_atr_image, (resize_width * 2, resize_height))
    combined_image.paste(mask_image, (resize_width * 3, resize_height))

    # Save the combined image in the validation folder with the file name
    output_path = os.path.join(output_dir, f"{base_file_name}_combined.jpg")
    combined_image.save(output_path)

    print(f"Combined image saved at: {output_path}")


Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_1_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_109_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_110_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_111_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_114_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_115_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_116_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_119_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\bottom_12_combined.jpg
Combined imag

In [7]:
import requests
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
import base64
from PIL import Image, ImageOps
from io import BytesIO
import os
import subprocess
import requests
# Base64 인코딩 예제 데이터 (추후 실제 DB와 연동 가능)
def encode_image_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return base64.b64encode(response.content).decode('utf-8')
        else:
            return f"Failed to fetch image from URL: {url}"
    except Exception as e:
        return f"Error fetching image from URL: {str(e)}"

def encode_image_from_file(file_path):
    try:
        with open(file_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"Error reading file: {str(e)}"

# 테스트용 Base64 데이터
clothing_url = "https://image-cdn.trenbe.com/product-images/1728547221306_5b7ac44bd35eb6a24d8039252d0b66e0_0.jpeg"
clothing_base64 = encode_image_from_url(clothing_url)

person_image_path = r"C:\Users\epdgn\Downloads\catvton_test\trenbe\test\images\bottom_1.jpg"
person_base64 = encode_image_from_file(person_image_path)


In [8]:
person_base64

'...'

In [9]:
clothing_base64

'...'